In [ ]:
!pip install transformers==4.41.2 datasets==2.18.0 accelerate==0.30.1 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 100.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.9/170.9 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 124.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.2.0 which is incompatible.


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AlbertTokenizer, AlbertForSequenceClassification, AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score
import re

LOAD TRAIN AND TEST


In [ ]:
train_df = pd.read_csv('/content/train.csv')
test_df  = pd.read_csv('/content/test.csv')

train_df.head(), test_df.head()


(      id          header_review  \
 0  50201             Luar Biasa   
 1  93053  Recomended Bagus Bang   
 2  28246   Barangnya Udh Sampai   
 3  13659    Barang Sudah Sampai   
 4   7886  Barang Sesuai Pesanan   
 
                                review_sangat_singkat  label  
 0  Thank u gan.sudah sampai.sesuai pesanan mantap...      1  
 1  Respon cepat. Recomended gan..thanks packing r...      1  
 2      Barangnya sangat bagus dan bahannya jga bagus      1  
 3                    Terima kasih gaaaaaaannnnnnnnnn      1  
 4                     Barang bagus,sesuai foto, puas      1  ,
       id                       header_review  \
 0  51890               Barang Sudah Sampai..   
 1  29962  Mantap Gan Pengiriman Cepat Sampai   
 2  86194              Sip...................   
 3  34344                               Trims   
 4  53163                             Kecewa!   
 
                                review_sangat_singkat  label  
 0                              barang sudah

Preprocessing


In [ ]:
def preprocess(text):
    text = str(text).lower()
    text = re.sub(r'[^a-zA-Z0-9\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

train_df['clean_review'] = train_df['review_sangat_singkat'].apply(preprocess)
test_df['clean_review']  = test_df['review_sangat_singkat'].apply(preprocess)


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("albert-base-v2")
config = AutoConfig.from_pretrained("albert-base-v2", num_labels=2)
bert = AutoModel.from_pretrained("albert-base-v2", config=config)


tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

TypeError: not a string

Dataset Class (disesuaikan dengan dataset kamu)

In [ ]:
class TextDataset(Dataset):
    def __init__(self, df, tokenizer, max_len=128):
        self.df = df
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        text = str(self.df.iloc[idx]["review_sangat_singkat"])
        label = int(self.df.iloc[idx]["label"])

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            return_tensors="pt"
        )

        return {
            "input_ids": encoding["input_ids"].flatten(),
            "attention_mask": encoding["attention_mask"].flatten(),
            "labels": torch.tensor(label, dtype=torch.long)
        }


In [ ]:
train_dataset = ReviewDataset(train_df)
test_dataset  = ReviewDataset(test_df)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=16)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)



Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from tqdm.auto import tqdm

EPOCHS = 4

for epoch in range(EPOCHS):
    print(f"\n===== EPOCH {epoch+1}/{EPOCHS} =====")
    model.train()
    total_loss = 0

    for batch in tqdm(train_loader, desc=f"Training Epoch {epoch+1}"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )

        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Loss Epoch {epoch+1}: {total_loss/len(train_loader):.4f}")



===== EPOCH 1/4 =====


Training Epoch 1:   0%|          | 0/4215 [00:00<?, ?it/s]

Loss Epoch 1: 0.2619

===== EPOCH 2/4 =====


Training Epoch 2:   0%|          | 0/4215 [00:00<?, ?it/s]

Loss Epoch 2: 0.2265

===== EPOCH 3/4 =====


Training Epoch 3:   0%|          | 0/4215 [00:00<?, ?it/s]

Loss Epoch 3: 0.2061

===== EPOCH 4/4 =====


Training Epoch 4:   0%|          | 0/4215 [00:00<?, ?it/s]

Loss Epoch 4: 0.1917


In [ ]:
def evaluate(model, loader):
    model.eval()
    preds = []
    trues = []

    with torch.no_grad():
        for batch in loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids, attention_mask)
            logits = outputs.logits

            pred = torch.argmax(logits, dim=1)

            preds.extend(pred.cpu().numpy())
            trues.extend(labels.cpu().numpy())

    return trues, preds


y_true, y_pred = evaluate(model, test_loader)
print("Akurasi:", accuracy_score(y_true, y_pred))
print("\nClassification Report:\n")
print(classification_report(y_true, y_pred, target_names=["Negatif", "Positif"]))


Akurasi: 0.915741029101353

Classification Report:

              precision    recall  f1-score   support

     Negatif       0.70      0.58      0.64      3655
     Positif       0.94      0.96      0.95     25244

    accuracy                           0.92     28899
   macro avg       0.82      0.77      0.79     28899
weighted avg       0.91      0.92      0.91     28899



In [ ]:
df_test = test_df.reset_index(drop=True)

print("\n===== 10 Contoh Prediksi ALBERT =====\n")

for i in range(10):
    teks = df_test.iloc[i]["review_sangat_singkat"]
    label_asli = y_true[i]
    pred = y_pred[i]
    status = "BENAR" if label_asli == pred else "SALAH"

    print(f"Teks     : {teks}")
    print(f"Asli     : {label_asli} ({'Negatif' if label_asli==0 else 'Positif'})")
    print(f"Prediksi : {pred} ({'Negatif' if pred==0 else 'Positif'})")
    print(f"Hasil    : {status}")
    print("-"*50)



===== 10 Contoh Prediksi ALBERT =====

Teks     : barang sudah sampai..
Asli     : 1 (Positif)
Prediksi : 1 (Positif)
Hasil    : BENAR
--------------------------------------------------
Teks     : mantap gan pengiriman cepat sampai
Asli     : 1 (Positif)
Prediksi : 1 (Positif)
Hasil    : BENAR
--------------------------------------------------
Teks     : Sip...................
Asli     : 1 (Positif)
Prediksi : 1 (Positif)
Hasil    : BENAR
--------------------------------------------------
Teks     : Barang Cepat sampai dan lumayan untuk harga sesuai tsb
Asli     : 1 (Positif)
Prediksi : 1 (Positif)
Hasil    : BENAR
--------------------------------------------------
Teks     : ukuran kekecilan
Asli     : 0 (Negatif)
Prediksi : 1 (Positif)
Hasil    : SALAH
--------------------------------------------------
Teks     : terriimaakaasihhh, bermanfaat dan dapat dipergunakan dengan kebutuhan lain
Asli     : 1 (Positif)
Prediksi : 1 (Positif)
Hasil    : BENAR
----------------------------------